# Decision Tree Regression

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [4]:
data =pd.read_csv('train.csv')
z=pd.read_csv('test.csv')
x_train =data
x_test =z

In [97]:
ytrain =data.Survived
ytrain.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [ ]:
x_train.head()

In [5]:

x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


**Clearing Null values**


In [141]:
x_train.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
title          0
dtype: int64

In [35]:
data['title']=0
for i in data:
  data['title']=data.Name.str.extract('([A-Za-z]+)\.')
data.title.nunique()

17

In [36]:
pd.crosstab(data.title,data.Sex).T

title,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


In [37]:
data.title.replace(['Capt',	'Col','Countess',	'Don','Dr',	'Jonkheer','Lady','Major','Master','Miss','Mlle','Mme','Mr','Mrs','Ms','Rev','Sir'],
                   ['Mr','Other','Mrs',   'Mr', 'Mr',  'Other', 'Mrs',    'Mr', 'Master','Miss','Miss','Miss','Mr','Mrs','Miss','Other','Mr'],inplace =True)


In [41]:
pd.crosstab(data.title,data.Sex).T.style.background_gradient(cmap='summer')

title,Master,Miss,Mr,Mrs,Other
Sex,,,,,
female,0,186,1,127,0
male,40,0,528,0,9


In [42]:
data.groupby('title').Age.mean()

title
Master     4.574167
Miss      21.860000
Mr        32.739609
Mrs       35.981818
Other     45.888889
Name: Age, dtype: float64

In [46]:
data.loc[(data.Age.isnull())&(data.title=='Mr'),'Age']=33
data.loc[(data.Age.isnull())&(data.title=='Mrs'),'Age']=36
data.loc[(data.Age.isnull())&(data.title=='Miss'),'Age']=22
data.loc[(data.Age.isnull())&(data.title=='Master'),'Age']=5
data.loc[(data.Age.isnull())&(data.title=='Other'),'Age']=46

In [48]:
data.Age.isnull().any()

False

In [49]:
data.Embarked.fillna('S',inplace=True)
data.Embarked.isnull().any()

False

In [ ]:
data.isnull().sum()

In [52]:
z.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

data['title']=0
for i in data:
  data['title']=data.Name.str.extract('([A-Za-z]+)\.')
data.title.nunique()

In [60]:
z['title']=0
for i in z:
  z['title'] =z.Name.str.extract('([A-Za-z]+)\.')
  z.title.nunique()

In [ ]:
z.title

In [65]:
z.title.replace(['Capt',	'Col','Countess',	'Don','Dr',	'Jonkheer','Lady','Major','Master','Miss','Mlle','Mme','Mr','Mrs','Ms','Rev','Sir','Dona'],
                   ['Mr','Other','Mrs',   'Mr', 'Mr',  'Other', 'Mrs',    'Mr', 'Master','Miss','Miss','Miss','Mr','Mrs','Miss','Other','Mr','Other'],inplace =True)

In [67]:
pd.crosstab(z.title,z.Sex).T

title,Master,Miss,Mr,Mrs,Other
Sex,,,,,
female,0,79,0,72,1
male,21,0,241,0,4


In [68]:
z.loc[(z.Age.isnull())&(z.title=='Mr'),'Age']=33
z.loc[(z.Age.isnull())&(z.title=='Mrs'),'Age']=36
z.loc[(z.Age.isnull())&(z.title=='Miss'),'Age']=22
z.loc[(z.Age.isnull())&(z.title=='Master'),'Age']=5
z.loc[(z.Age.isnull())&(z.title=='Other'),'Age']=46

In [ ]:
z.isnull().any()

In [ ]:
x_train =x_train.drop(columns=['Name','Ticket','Cabin','Survived'])

In [76]:
x_test =x_test.drop(columns=['Name','Ticket','Cabin'])


In [81]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         418 non-null    float64
 7   Embarked     418 non-null    object 
 8   title        418 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 29.5+ KB


In [78]:
x_test.Fare.fillna(value=0,inplace=True)

In [80]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Embarked     891 non-null    object 
 8   title        891 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


In [82]:
x_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,title
0,892,3,male,34.5,0,0,7.8292,Q,Mr
1,893,3,female,47.0,1,0,7.0000,S,Mrs
2,894,2,male,62.0,0,0,9.6875,Q,Mr
3,895,3,male,27.0,0,0,8.6625,S,Mr
4,896,3,female,22.0,1,1,12.2875,S,Mrs


In [84]:
x_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,title
0,1,3,male,22.0,1,0,7.2500,S,Mr
1,2,1,female,38.0,1,0,71.2833,C,Mrs
2,3,3,female,26.0,0,0,7.9250,S,Miss
3,4,1,female,35.0,1,0,53.1000,S,Mrs
4,5,3,male,35.0,0,0,8.0500,S,Mr


In [85]:
x_train_label =x_train
x_test_label = x_test

**Working on Categorical Data**

> Indented block



In [87]:
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()
x_train_label.Sex =le.fit_transform(x_train_label.Sex)
x_test_label.Sex =le.fit_transform(x_test_label.Sex)

In [88]:
x_train_label.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,title
0,1,3,1,22.0,1,0,7.2500,S,Mr
1,2,1,0,38.0,1,0,71.2833,C,Mrs
2,3,3,0,26.0,0,0,7.9250,S,Miss
3,4,1,0,35.0,1,0,53.1000,S,Mrs
4,5,3,1,35.0,0,0,8.0500,S,Mr


In [89]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [7,8])], remainder='passthrough')
x_train_label = np.array(ct.fit_transform(x_train_label))
x_test_label =np.array(ct.fit_transform(x_test_label))

In [94]:
traindata =pd.DataFrame(x_train_label)
traindata.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,1.0,22.0,1.0,0.0,7.2500
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,38.0,1.0,0.0,71.2833
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,26.0,0.0,0.0,7.9250
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,0.0,35.0,1.0,0.0,53.1000
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,5.0,3.0,1.0,35.0,0.0,0.0,8.0500


In [95]:
testdata=pd.DataFrame(x_test_label)
testdata.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,892.0,3.0,1.0,34.5,0.0,0.0,7.8292
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,893.0,3.0,0.0,47.0,1.0,0.0,7.0000
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,894.0,2.0,1.0,62.0,0.0,0.0,9.6875
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,895.0,3.0,1.0,27.0,0.0,0.0,8.6625
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,896.0,3.0,0.0,22.0,1.0,1.0,12.2875


In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(traindata, ytrain, test_size = 1/3, random_state = 0)

## Training the Decision Tree Regression model on the whole dataset

In [142]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(traindata, ytrain)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

## Predicting a new result

In [134]:
y_test =regressor.predict(testdata)

In [135]:
y_test

array([1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 1., 0.

In [ ]:
ytest =y_test.copy()
ytest

In [140]:
output=pd.DataFrame({'PassengerID':x_test.PassengerId,'Survived':ytest})
output.to_csv('my_submission.csv',index=False)

In [127]:
ytest['PassengerID'] =x_test.PassengerId

## Visualising the Decision Tree Regression results (higher resolution)

In [ ]:
X_grid = np.arange(min(traindata), max(traindata), 0.01)
X_grid = X_grid.reshape((len(X_grid), 1))
plt.scatter(traindata, ytrain, color = 'red')
plt.plot(X_grid, regressor.predict(X_grid), color = 'blue')
plt.title('Truth or Bluff (Decision Tree Regression)')
plt.xlabel('Position level')
plt.ylabel('Salary')
plt.show()